In [1]:
import pandas as pd
import numpy as np
import json
import ast
import re
import os
import pickle
from tqdm import tqdm
from constants import base_path, model_list, max_n_questions
import matplotlib.pyplot as plt
tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
tqdm.pandas()

c:\Users\myras\Documents\MYMY\ACADS\AY 2023-2024 2ND SEM\STAT299 - Capstone\Repo\VITAS\vitas-env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
with open(f"{base_path}\\input\\release_conditions.json") as f:
  disease_dict = json.load(f)
disease_list = list(disease_dict.keys())

In [3]:
with open(f"{base_path}\\input\\release_evidences.json") as f:
  evidences = json.load(f)
evidences_list = []
evidences_dict = {}
evidences_en_to_code = {}
for e in evidences.keys():
  # only binary symptoms and antecedents
  if (not evidences[e]["possible-values"]):
    evidences_list.append(e)
    evidences_dict[e] = evidences[e]["question_en"]
    evidences_en_to_code[evidences[e]["question_en"]] = e
evidences_code_to_en = evidences_dict
evidences_list_en = list(evidences_en_to_code.keys())
evidences_dict["AGE"] = "AGE"
evidences_dict["SEX"] = "SEX"
feature_columns = ["AGE", "SEX"] + evidences_list

In [4]:
def get_next_question(evidences, questionnaire, feature_embeddings_df):
    centroid = np.array([feature_embeddings_df.loc[e].values for e in evidences]).mean(axis=0)
    _, indices = questionnaire.kneighbors([centroid])
    ask_list = [evidences_list_en[i] for i in indices[0] if evidences_list_en[i] not in evidences]
    try:
        return ask_list
    except:
        return []

In [5]:
def get_evidences(answers, user_evidences, questionnaire, feature_embeddings_df, n_questions):
    ask = True
    question_counter = 1 # counts initial evidence sa q0
    evidences_en = [evidences_code_to_en[e] for e in user_evidences]
    answers_en = [evidences_code_to_en[e] for e in answers if e in evidences_list]
    asked = evidences_en.copy()
    while question_counter < n_questions:
      ask =  True
      next_question_idx = 0
      next_question = get_next_question(evidences_en, questionnaire, feature_embeddings_df)
      while ask and (question_counter < n_questions):
        if next_question[next_question_idx] not in asked:
          answer = 1 if next_question[next_question_idx] in answers_en else 0
          asked.append(next_question[next_question_idx])
          question_counter+=1
          if answer==1:
            evidences_en.append(next_question[next_question_idx])
            ask = False
          else:
            next_question_idx += 1
          if next_question_idx > n_questions:
            break
        else:
            next_question_idx += 1
    return [evidences_en_to_code[e] for e in evidences_en]

In [6]:
def data_proc(df, questionnaire, feature_embeddings_df, n_questions):
    df["binary_evidences_all"] = df["EVIDENCES"].apply(lambda x: [d for d in ast.literal_eval(x) if d in evidences_list])
    df["binary_evidences_all_count"] = df["binary_evidences_all"].apply(lambda x: len(x))
    df["binary_evidences"] = df[["EVIDENCES", "INITIAL_EVIDENCE"]].progress_apply(lambda x: get_evidences(ast.literal_eval(x[0]), [x[1]], questionnaire, feature_embeddings_df, n_questions), axis=1)
    df["binary_evidences_count"] = df["binary_evidences"].apply(lambda x: len(x))
    df["hit_rate"] = df["binary_evidences_count"]/df["binary_evidences_all_count"]
    hit_rate = df["hit_rate"]
    for e in evidences_list:
        df[e] = df["binary_evidences"].apply(lambda x: 1 if e in x else 0)
    df["SEX"] = df["SEX"].map({'F': 0, 'M': 1})
    ftr_df = df[feature_columns + ["PATHOLOGY"]]
    questionnaire_df = df[["binary_evidences_all", "binary_evidences", "INITIAL_EVIDENCE"]]
    return ftr_df, hit_rate, questionnaire_df

In [7]:
def pred(x):
    pred_list = []
    for i in range(len(disease_list)):
        if x[i] > 0:
            pred_list.append({
                "disease": disease_list[i],
                "probability": x[i]})
    if pred_list:
        pred_df = pd.DataFrame(pred_list).set_index('disease')
        # return only top 1 - allows ties
        pred_df['rank'] = pred_df['probability'].rank(method='min', ascending=False)
        pred_df = pred_df.sort_values(by="rank")
        pred_df = pred_df[pred_df["rank"]<=1][["probability"]]
        return list(pred_df.index)
    else:
        return []

In [8]:
def get_missed_evidence(actual, asked):
    return list(set(actual)^set(asked))

In [9]:
def validate(x):
    return [x[0]]==x[1]

In [10]:
with open(f'{base_path}\\output\\questionnaire\\questionnaire.pkl', 'rb') as f:
    questionnaire = pickle.load(f)
feature_embeddings_df = pd.read_pickle(f'{base_path}\\output\\questionnaire\\questionnaire_embeddings.pkl')

In [11]:
diagnosis_df_valid = pd.read_csv(f"{base_path}\\input\\release_validate_patients")

In [12]:
# test
data_proc(diagnosis_df_valid.head(5), questionnaire, feature_embeddings_df, n_questions=20)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00, 89.81it/s]


(   AGE  SEX  E_91  E_53  E_159  E_129  E_154  E_155  E_210  E_140  ...  E_199  \
 0   55    0     0     1      0      0      1      0      0      1  ...      0   
 1   10    0     0     1      0      0      0      1      0      0  ...      0   
 2   68    0     1     1      0      1      0      0      0      0  ...      0   
 3   13    1     0     1      0      0      0      0      0      0  ...      0   
 4   48    1     0     1      0      0      0      0      1      0  ...      0   
 
    E_121  E_120  E_142  E_195  E_183  E_224  E_223  E_5     PATHOLOGY  
 0      0      0      0      0      0      0      0    0        Anemia  
 1      0      0      0      0      0      0      0    0  Panic attack  
 2      0      0      0      0      0      0      0    0     Influenza  
 3      0      0      0      0      0      0      0    0        Anemia  
 4      0      0      0      0      0      0      0    0     Boerhaave  
 
 [5 rows x 211 columns],
 0    1.000000
 1    0.933333
 2    1.000

In [13]:
questionnaire_dict = {}
n_questions_list = list(range(5, max_n_questions, 5))
if max_n_questions not in n_questions_list:
    n_questions_list.append(max_n_questions)

In [14]:
data_path = f"{base_path}\\processed\\questionnaire_experiments"
if not os.path.exists(data_path):
    os.makedirs(data_path)

# is_load_data default is False
# if data is already available in processed folder, change to True
is_load_data = False
    
for n_questions in n_questions_list:
    diagnosis_df_valid = pd.read_csv(f"{base_path}\\input\\release_validate_patients")
    print(f"Asking {n_questions} questions...")
    if not is_load_data:
        diagnosis_df_valid, hit_rate, questionnaire_df = data_proc(diagnosis_df_valid, questionnaire, feature_embeddings_df, n_questions)
        with open(f"{data_path}\\diagnosis_df_valid_{n_questions}.pkl",'wb') as f:
            pickle.dump(diagnosis_df_valid, f)
        with open(f"{data_path}\\hit_rate_{n_questions}.pkl",'wb') as f:
            pickle.dump(hit_rate, f)
    else:
        with open(f'{data_path}\\diagnosis_df_valid_{n_questions}.pkl', 'rb') as f:
            diagnosis_df_valid = pickle.load(f)
        with open(f'{data_path}\\hit_rate_{n_questions}.pkl', 'rb') as f:
            hit_rate = pickle.load(f)
    questionnaire_dict[f"questionnaire_data_{n_questions}"] = {
        "diagnosis_df_valid": diagnosis_df_valid,
        "hit_rate": hit_rate,
    }


Asking 5 questions...
Asking 10 questions...
Asking 15 questions...
Asking 20 questions...
Asking 25 questions...
Asking 30 questions...
Asking 35 questions...
Asking 40 questions...
Asking 45 questions...
Asking 50 questions...
Asking 55 questions...
Asking 60 questions...
Asking 65 questions...
Asking 70 questions...
Asking 75 questions...
Asking 80 questions...
Asking 85 questions...
Asking 90 questions...
Asking 95 questions...
Asking 100 questions...
Asking 105 questions...
Asking 110 questions...
Asking 115 questions...
Asking 120 questions...
Asking 125 questions...
Asking 130 questions...
Asking 135 questions...
Asking 140 questions...
Asking 145 questions...
Asking 150 questions...
Asking 155 questions...
Asking 160 questions...
Asking 165 questions...
Asking 170 questions...
Asking 175 questions...
Asking 180 questions...
Asking 185 questions...
Asking 190 questions...
Asking 195 questions...
Asking 200 questions...
Asking 205 questions...
Asking 208 questions...


In [15]:
metrics_dict = {model_name:{} for model_name in model_list["tree-based"]}
metrics_dict["logistic_regression"] = {}

In [16]:
experiment_path = f"{base_path}\\output\\error_analysis_questionnaire"
if not os.path.exists(experiment_path):
    os.makedirs(experiment_path)

In [17]:
def experiment(model_name):
    print(f"Evaluating {model_name}...")
    model_dict = {}
    for disease in disease_list:
        disease_filename = re.sub('[^a-zA-Z0-9 \n\.]', '', disease).replace(" ", "_")
        with open(f'{base_path}\\output\\diseases\\{disease_filename}\\{model_name}\\{disease_filename}_model.pkl', 'rb') as f:
            model_dict[disease] = pickle.load(f)
    for n_questions in n_questions_list:
        diagnosis_df_valid = questionnaire_dict[f"questionnaire_data_{n_questions}"]["diagnosis_df_valid"]
        hit_rate = questionnaire_dict[f"questionnaire_data_{n_questions}"]["hit_rate"]
        print(f"using {n_questions} questions predicting diagnosis...")
        for disease in disease_list:
            clf_model = model_dict[disease]
            diagnosis_df_valid[disease] = clf_model.predict_proba(diagnosis_df_valid[feature_columns])[:,1]
        diagnosis_df_valid["predicted_diagnosis"] = diagnosis_df_valid[disease_list].progress_apply(lambda x: pred(x), axis=1)
        diagnosis_df_valid["is_matched"] = diagnosis_df_valid[["PATHOLOGY", "predicted_diagnosis"]].progress_apply(lambda x: validate(x), axis=1)
        diagnosis_df_valid["hit_rate"] = hit_rate
        print("validating...")
        acc = diagnosis_df_valid.is_matched.value_counts().to_dict()
        acc["error_rate"] = acc[False]/len(diagnosis_df_valid)
        acc["mean_hit_rate"] = np.mean(hit_rate)
        hit_rate_misdiagnosed = diagnosis_df_valid[diagnosis_df_valid["is_matched"]==False]["hit_rate"]
        acc["mean_hit_rate_misdiagnosed"] = np.mean(hit_rate_misdiagnosed)
        metrics_dict[model_name][f"n_questions_{n_questions}"] = acc      
    metrics_dict[model_name]["plt_y"] = [metrics_dict[model_name][f"n_questions_{i}"]["error_rate"] for i in n_questions_list]
    if not os.path.exists(f"{experiment_path}\\{model_name}"):
        os.makedirs(f"{experiment_path}\\{model_name}")
    with open(f"{experiment_path}\\{model_name}\\validation_metric_questionnaire_experiments.json", "w") as outfile: 
        json.dump(metrics_dict[model_name], outfile, indent=True)
    

## Tree-based models

In [18]:
for model_name in model_list["tree-based"]:
    experiment(model_name)
# 947m 50 s

Evaluating decision_tree...
using 5 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:02<00:00, 47621.69it/s]


validating...
using 10 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:02<00:00, 46138.18it/s]


validating...
using 15 questions predicting diagnosis...


...

validating...
using 205 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:01<00:00, 125741.81it/s]


validating...
using 208 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:01<00:00, 74024.13it/s]


validating...


## Logistic Regression

In [22]:
experiment("logistic_regression")

Evaluating logistic_regression...
using 5 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:01<00:00, 122277.84it/s]


validating...
using 10 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:01<00:00, 126940.70it/s]


validating...
using 15 questions predicting diagnosis...


...

validating...
using 205 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:01<00:00, 114979.03it/s]


validating...
using 208 questions predicting diagnosis...


100%|██████████| 132448/132448 [00:01<00:00, 116037.13it/s]


validating...


In [24]:
with open(f'{base_path}\\output\\error_analysis_questionnaire\\logistic_regression\\validation_metric_questionnaire_experiments.json', 'rb') as f:
    metrics_dict["logistic_regression"] = json.load(f)
metrics_dict

{'decision_tree': {'n_questions_5': {False: 94080,
   True: 38368,
   'error_rate': 0.7103165015704277,
   'mean_hit_rate': 0.33105369578602123,
   'mean_hit_rate_misdiagnosed': 0.2886861233365244},
  'n_questions_10': {False: 77696,
   True: 54752,
   'error_rate': 0.5866151244261899,
   'mean_hit_rate': 0.5348788121386866,
   'mean_hit_rate_misdiagnosed': 0.44139503255762713},
  'n_questions_15': {True: 79010,
   False: 53438,
   'error_rate': 0.403464000966417,
   'mean_hit_rate': 0.7015726661850556,
   'mean_hit_rate_misdiagnosed': 0.5291734348047398},
  'n_questions_20': {True: 97850,
   False: 34598,
   'error_rate': 0.26121949746315537,
   'mean_hit_rate': 0.8109909153294077,
   'mean_hit_rate_misdiagnosed': 0.5816529367222822},
  'n_questions_25': {True: 108763,
   False: 23685,
   'error_rate': 0.1788248973181928,
   'mean_hit_rate': 0.8810154199870976,
   'mean_hit_rate_misdiagnosed': 0.6377917163772117},
  'n_questions_30': {True: 112857,
   False: 19591,
   'error_rate': 0.

In [29]:
for model_name in model_list["tree-based"]:
    plt.plot(n_questions_list, metrics_dict[model_name]["plt_y"], marker='o', label = model_name, alpha=0.5)
plt.plot(n_questions_list, metrics_dict["logistic_regression"]["plt_y"], marker='o', label = "logistic_regression", alpha=0.5)
plt.xlabel('number of questions')
plt.ylabel('misdiagnosis rate')
plt.title('Questionnaire Experiments')
plt.ylim(0, 0.8)
plt.legend()
plt.savefig(f'{base_path}\\output\\error_analysis_questionnaire\\questionnaire_experiments.jpg', bbox_inches='tight')
plt.clf()

<Figure size 640x480 with 0 Axes>